In [1]:
from gw_signal_tools.inner_product import inner_product, norm, overlap
from gw_signal_tools.PSDs import psd_gw150914

import astropy.units as u
import lalsimulation.gwsignal.core.waveform as wfm

/home/user/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(True)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


The pyseobnr package has failed to load, you will not be able to employ SEOBNRv5 approximants.


# Inner Product Verifications

First we need example signals

In [2]:
# Define the dictionary
# deltaT = 1./1024.*u.s
# deltaT = 1./2048.*u.s
deltaT = 1./4096.*u.s
f_min = 20.*u.Hz  # Cutoff frequency
f_ref = 20.*u.Hz  # Frequency where we specify spins
distance = 440.*u.Mpc
inclination = 2.7*u.rad  # Value taken from posteriors.ipynb, where posterior of inclination is plotted
phiRef = 0.*u.rad
eccentricity = 0.*u.dimensionless_unscaled
longAscNodes = 0.*u.rad
meanPerAno = 0.*u.rad


parameters_gw150914 = {
    'mass1' : 36*u.solMass,
    'mass2' : 29*u.solMass,
    'deltaT' : deltaT,
    # 'deltaF' : deltaT / u.s * u.Hz,  # Does work
    'f22_start' : f_min,
    'f22_ref': f_ref,
    'phi_ref' : phiRef,
    'distance' : distance,
    'inclination' : inclination,
    'eccentricity' : eccentricity,
    'longAscNodes' : longAscNodes,
    'meanPerAno' : meanPerAno,
    'condition' : 0
}

# Define the approximant
approximant = 'IMRPhenomXPHM'

# Call the generator
gen = wfm.LALCompactBinaryCoalescenceGenerator(approximant)


# Generate waveform -> output are two time series, h_plus and h_cross
hp_t, hc_t = wfm.GenerateTDWaveform(parameters_gw150914, gen)

# Two waveforms will be generated in frequency domain, first with finer
# sampling and then with coarser one
hp_f_fine, hc_f_fine = wfm.GenerateFDWaveform(parameters_gw150914, gen)

hp_f_coarse, hc_f_coarse = wfm.GenerateFDWaveform(parameters_gw150914 | {'deltaF': 1.0 / (hp_t.size * hp_t.dx)}, gen)

/home/user/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/lalsimulation/gwsignal/core/waveform.py:226: UserWarning: This code is currently UNREVIEWED, use with caution!
  warnings.warn("This code is currently UNREVIEWED, use with caution!")


To show the difference in sampling, let us look at the values

In [16]:
print(hp_f_coarse.df, hp_f_fine.df)

0.5838916607270136 Hz 0.0625 Hz


This is almost one order of magnitude difference

In [3]:
# df_test = hp_f_coarse.df / 2  # Without 1/2 assertion error
# df_test = 0.125  # Testing of df parameter
# df_test = 0.125 * u.Hz  # Testing of df parameter -> results not even too bad
df_test = None
# f_range_test = [f_min, 250]  # Activating makes results slightly more unequal
# f_range_test = [0, 250000]  # Testing of range parameter
f_range_test = None

print(norm(hp_t, psd_gw150914, df=df_test, f_range=f_range_test))
print(norm(hp_f_fine, psd_gw150914, df=df_test, f_range=f_range_test))
print(norm(hp_f_coarse, psd_gw150914, df=df_test, f_range=f_range_test))

29.753670381158557
29.75382206975821
29.632756613079803


In [15]:
print(overlap(hp_t, hp_f_coarse, psd_gw150914))
print(overlap(hp_t, hp_f_fine, psd_gw150914))

0.9996911301802703
0.9999994139733958


As we can see, sampling of input makes a notable difference because the quality of interpolation depends on that.

Nonetheless, all results are consistent and even the ones for coarser sampling are acceptable.

## Benchmarking

Testing which method is most efficient

In [4]:
# Set test parameters
df_test = None
f_range_test = None

In [5]:
%%timeit

norm(hp_t, psd_gw150914, df=df_test, f_range=f_range_test)

9.07 ms ± 41 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit

norm(hp_t, psd_gw150914, df=0.125, f_range=f_range_test)

6.45 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit

norm(hp_f_fine, psd_gw150914, df=df_test, f_range=f_range_test)

3.19 ms ± 13.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit

norm(hp_f_coarse, psd_gw150914, df=df_test, f_range=f_range_test)

3.44 ms ± 28.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Clearly, choice of df makes bigger difference for time domain waveform, as opposed to frequency domain one. However, one should remember that generating with finer resolution also takes more time. So let us now see which method is fastest overall

In [9]:
%%timeit

hp_t, _ = wfm.GenerateTDWaveform(parameters_gw150914, gen)

norm(hp_t, psd_gw150914, df=df_test, f_range=f_range_test)

193 ms ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit

hp_t, _ = wfm.GenerateTDWaveform(parameters_gw150914, gen)

norm(hp_t, psd_gw150914, df=0.125, f_range=f_range_test)

204 ms ± 9.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit

hp_f_fine, _ = wfm.GenerateFDWaveform(parameters_gw150914, gen)

norm(hp_f_fine, psd_gw150914, df=df_test, f_range=f_range_test)

201 ms ± 7.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit

hp_f_coarse, _ = wfm.GenerateFDWaveform(parameters_gw150914 | {'deltaF': 1.0 / (hp_t.size * hp_t.dx)}, gen)

norm(hp_f_coarse, psd_gw150914, df=df_test, f_range=f_range_test)

196 ms ± 7.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
